# 回転の推定I:等方性誤差

In [1]:
from pathlib import Path
import sys

from itkwidgets import view
import numpy as np
from scipy import linalg
from scipy.stats import special_ortho_group
from scipy.spatial.transform import Rotation

sys.path.append('../libs')
import util

In [2]:
A = util.load_point_cloud(Path('../bunny/data/bun180.ply').resolve())

In [3]:
view(point_sets=A)

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [4]:
R = special_ortho_group.rvs(3)
print(R)

[[ 0.14200882 -0.98849183 -0.05212857]
 [-0.14404829  0.03146479 -0.9890703 ]
 [ 0.97932813  0.14796574 -0.13792228]]


In [5]:
noise = np.random.normal(0, 3e-3, A.shape)
A_prime = A @ R.T + noise

In [6]:
view(point_sets=[A, A_prime])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

## 4.3 特異値分解による解法

In [7]:
estimated_R = util.estimate_R_using_SVD(A, A_prime)
print(util.eval_R_error(estimated_R, R))

0.0005689624432243011


In [8]:
view(point_sets=[A @ estimated_R.T, A_prime])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

## 4.4 四元数表示による解法

In [9]:
N = A.T @ A_prime
N_tilde = np.array([
    [
        N[0, 0] + N[1, 1] + N[2, 2],
        -N[2, 1] + N[1, 2],
        N[2, 0] - N[0, 2],
        -N[1, 0] + N[0, 1],
    ], [
        -N[2, 1] + N[1, 2],
        N[0, 0] - N[1, 1] - N[2, 2],
        N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
    ], [
        N[2, 0] - N[0, 2],
        N[1, 0] + N[0, 1],
        -N[0, 0] + N[1, 1] - N[2, 2],
        N[2, 1] + N[1, 2],
    ], [
        -N[1, 0] + N[0, 1],
        N[2, 0] + N[0, 2],
        N[2, 1] + N[1, 2],
        -N[0, 0] - N[1, 1] + N[2, 2],
    ],
])

In [10]:
w, v = linalg.eig(N_tilde)
max_vec = v[:, np.argmax(w)]
# 書籍では(w, x, y, z)だが，scipyでは(x, y, z, w)
r = Rotation.from_quat(max_vec[[1, 2, 3, 0]])
print(util.eval_R_error(r.as_dcm(), R))

0.000568962443223393


In [11]:
view(point_sets=[A @ r.as_dcm().T, A_prime])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

## 4.5 回転行列の最適補正

In [12]:
R_hat = R + np.random.normal(0, 1e-2, R.shape)
R_hat @ R_hat.T

array([[ 1.0140317 ,  0.02316698, -0.00761126],
       [ 0.02316698,  1.0199991 , -0.00955682],
       [-0.00761126, -0.00955682,  0.96885434]])

In [13]:
U, s, Vh = linalg.svd(R_hat)
V = Vh.T
corrected_R = U @ np.diag([1, 1, linalg.det(U @ V)]) @ Vh
corrected_R @ corrected_R.T

array([[1.00000000e+00, 2.00331120e-16, 6.00577469e-16],
       [2.00331120e-16, 1.00000000e+00, 3.54886553e-17],
       [6.00577469e-16, 3.54886553e-17, 1.00000000e+00]])